In [1]:
import requests
website='https://www.epa.gov/superfund/national-priorities-list-npl-sites-state'
r  = requests.get(website)

In [4]:
#scrape data
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
url =website

browser = webdriver.Chrome()
wait = WebDriverWait(browser, 10)
browser.get(url)
soup = BeautifulSoup(browser.page_source, "lxml")
soup.prettify()

'<!DOCTYPE html PUBLIC "-//W3C//DTD HTML+RDFa 1.1//EN">\n<!--[if IEMobile 7]><html class="iem7 no-js" lang="en" dir="ltr"><![endif]-->\n<!--[if lt IE 7]><html class="lt-ie9 lt-ie8 lt-ie7 no-js" lang="en" dir="ltr"><![endif]-->\n<!--[if (IE 7)&(!IEMobile)]><html class="lt-ie9 lt-ie8 no-js" lang="en" dir="ltr"><![endif]-->\n<!--[if IE 8]><html class="lt-ie9 no-js" lang="en" dir="ltr"><![endif]-->\n<!--[if (gt IE 8)|(gt IEMobile 7)]><!-->\n<html class="not-oldie js" dir="ltr" lang="en" xmlns="http://www.w3.org/1999/xhtml">\n <!--<![endif]-->\n <head profile="http://www.w3.org/1999/xhtml/vocab">\n  <meta charset="utf-8"/>\n  <meta content="false" http-equiv="ImageToolbar"/>\n  <meta content="on" http-equiv="cleartype"/>\n  <meta content="Drupal 7 (http://drupal.org)" name="generator"/>\n  <meta content="true" name="HandheldFriendly"/>\n  <link href="https://www.epa.gov/sites/all/themes/epa/favicon.ico" rel="shortcut icon" type="image/vnd.microsoft.icon"/>\n  <meta content="width" name="Mob

In [5]:
#splitting list
raw_list=[i.split('<tr><td>')[1] for i in str(soup).split('</span>')[11:] if '<tr><td>' in i]

raw_list[0]

'Alabama Army Ammunition Plant</td><td>Childersburg</td><td>AL6210020008</td><td>07/22/1987</td><td>36.83</td><td>Yes</td><td><ul><li><a href="https://semspub.epa.gov/src/document/04/11121411" target="_blank">Site Listing Narrative</a></li><li><a href="https://cumulis.epa.gov/supercpad/cursites/csitinfo.cfm?id=0400449" target="_blank">Site Progress Profile</a></li><li><a href="https://semspub.epa.gov/src/document/11/189629" target="_blank">Federal Register Notice (PDF)</a><span class="fileinfo">\n              (27 pp, 287 K)\n            '

In [45]:
#parsing out the individual elements
site_name=[i.split('</td><td>')[0] for i in raw_list]
city=[i.split('</td><td>')[1] for i in raw_list]
site_epa_id=[i.split('</td><td>')[2] for i in raw_list]
date_added=[i.split('</td><td>')[3] for i in raw_list]
site_score=[i.split('</td><td>')[4] for i in raw_list]
federal_facility_ind=[i.split('</td><td>')[5] for i in raw_list]

site_narrative_url=[i.split('href="')[1].split('" target')[0] if len(i.split('href="'))==4 else '' for i in raw_list]
site_progress_url=[i.split('href="')[2].split('" target')[0] if len(i.split('href="'))==4 else\
                   i.split('href="')[1].split('" target')[0] for i in raw_list]
federal_register_url=[i.split('href="')[3].split('" target')[0] if len(i.split('href="'))==4 else\
                      i.split('href="')[2].split('" target')[0] for i in raw_list]

In [66]:
#saving the data as csv
import pandas as pd
df=pd.DataFrame({'site_name':site_name,'city':city,'site_epa_id':site_epa_id,'date_added':date_added,'site_score':site_score\
             ,'federal_facility_ind':federal_facility_ind,'site_narrative_url':site_narrative_url,\
             'site_progress_url':site_progress_url,'federal_register_url':federal_register_url})\

#saving a list of narrative ids so that I can download them programmatically
narratives_locs=(df.site_narrative_url+'.pdf').str.replace("src/document",'work')
with open('data/narrative_locs.txt', 'w') as f:
    for nar in list(narratives_locs):
        if type(nar)==str:
            f.write(nar+"\n")

df.to_csv('./data/priorities_list.csv',index=False)